# 能量包络测试行星是否调制

使用“能量包络模型”作为筛选器，来系统性地检验哪些基础频率（残差中的信号）的能量，是真的受到了11年行星周期（拟合SSN）的调制。这个方法（高频信号的能量包络 vs 低频驱动力）在物理上比LGBM的“全频率梳”特征工程更具可解释性。我们将不再局限于20-35天，而是遍历所有相关的物理频带（自转、Rieger、年度、QBO），并量化它们各自与11年周期的“调制关系”。筛选调制频率的完整代码以下是完整的Python脚本。它会：定义一系列你感兴趣的物理频带（天）。循环遍历4个模型（M8+2, M0+3...）。对于每个模型，再循环遍历每个频带。核心步骤：提取该频带的能量包络，并（用一个简单的LGBM）拟合 $Y_{包络} = f(X_{行星拟合})$，在测试集上计算R²。最后汇总一张表，显示每个频带在4个模型上的平均R²，告诉您哪些频率“值得”被调制。

In [3]:
import pandas as pd
import numpy as np
import time
import warnings
import os
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from scipy.signal import butter, sosfiltfilt, hilbert
import lightgbm as lgb  # 我们用一个简单的LGBM作为检验模型 f

# 确保天文学库可用 (如果LGBM代码也在此运行)
try:
    import lightgbm as lgb
    print("LightGBM 导入成功。")
except ImportError:
    print("错误：未找到 lightgbm 库。")
    exit()

try:
    # 检查 scipy (信号处理库)
    import scipy
    print(f"Scipy 导入成功 (版本: {scipy.__version__})。")
except ImportError:
    print("错误: 本脚本需要 scipy 库。")
    print("请安装: pip install scipy")
    exit()

warnings.filterwarnings('ignore')

# --- 核心函数 1: 带通滤波器 ---
def bandpass_filter(data, lowcut_days, highcut_days, fs=1.0, order=4):
    """
    应用零相位带通滤波器
    fs=1.0 意味着采样间隔为1天
    """
    # 将周期（天）转换为频率（1/天）
    low_freq = 1.0 / highcut_days
    high_freq = 1.0 / lowcut_days
    
    # Nyquist 频率
    nyq = 0.5 * fs
    
    # 归一化频率
    low = low_freq / nyq
    high = high_freq / nyq
    
    # 确保频率在有效范围内
    low = max(low, 1e-9) 
    high = min(high, 0.9999)

    if low >= high:
        print(f"警告: 频率范围无效 [{low}, {high}]。跳过滤波。")
        return data # 返回原始数据

    # 创建滤波器
    sos = butter(order, [low, high], btype='band', output='sos')
    
    # 应用零相位滤波器
    filtered_data = sosfiltfilt(sos, data)
    return filtered_data

# --- 核心函数 2: 计算能量包络 ---
def get_energy_envelope(data, smoothing_window_days=365):
    """
    计算信号的能量包络 (希尔伯特变换)，并进行平滑
    """
    # 1. 计算解析信号
    analytic_signal = hilbert(data)
    
    # 2. 计算能量包络 (瞬时幅度)
    envelope = np.abs(analytic_signal)
    
    # 3. (关键) 对包络进行平滑，以观察其长期趋势
    #    我们使用中心滑动平均
    envelope_series = pd.Series(envelope)
    smoothed_envelope = envelope_series.rolling(
        window=smoothing_window_days, 
        center=True, 
        min_periods=int(smoothing_window_days * 0.1) # 允许边缘
    ).mean()
    
    return smoothed_envelope

# --- 核心函数 3: 运行调制检验 ---
def run_modulation_test(
    data_full, model_name, train_end_date, test_end_date, 
    band_name, freq_band_days, envelope_smoothing_days=365
):
    """
    检验一个频带(band)的能量包络是否被11年周期(拟合SSN)调制
    """
    
    target_residual_column = f'残差_{model_name}'
    planetary_fit_column = f'拟合SSN_{model_name}'

    print(f"  测试频带: {band_name} ({freq_band_days[0]}-{freq_band_days[1]} 天)")

    # 1. 准备数据
    data = data_full[['日期', target_residual_column, planetary_fit_column]].copy()
    
    # 筛选出用于分析的干净数据 (拟合SSN 和 残差 都必须存在)
    data_clean = data.dropna(subset=[target_residual_column, planetary_fit_column])
    if len(data_clean) < 5000: # 需要足够数据来滤波
        print(f"  警告: {model_name} 的干净数据不足 ({len(data_clean)}行)。")
        return None

    # --- 信号处理 ---
    
    # 2. Y (残差) -> 带通滤波
    Y_filtered = bandpass_filter(
        data_clean[target_residual_column].values, 
        freq_band_days[0], 
        freq_band_days[1]
    )
    
    # 3. Y_filtered -> 能量包络
    Y_envelope = get_energy_envelope(
        Y_filtered, 
        smoothing_window_days=envelope_smoothing_days
    )
    
    data_clean.loc[:, 'Y_Envelope'] = Y_envelope
    
    # 4. X (驱动力) -> 也进行平滑
    #    我们比较的是两个慢变量：包络的慢变化 vs 11年周期的慢变化
    data_clean.loc[:, 'X_Driver_Smooth'] = data_clean[planetary_fit_column].rolling(
        window=envelope_smoothing_days, 
        center=True, 
        min_periods=int(envelope_smoothing_days * 0.1)
    ).mean()

    # --- 建立检验模型 ---
    
    # 5. 再次清理 (平滑和滤波会产生新的NaN)
    data_model = data_clean.dropna(subset=['Y_Envelope', 'X_Driver_Smooth'])
    
    # 6. 定义 训练/测试 掩码
    train_mask = (data_model['日期'] <= train_end_date)
    test_mask = (data_model['日期'] > train_end_date) & (data_model['日期'] <= test_end_date)

    X_train = data_model.loc[train_mask, ['X_Driver_Smooth']]
    Y_train = data_model.loc[train_mask, 'Y_Envelope']
    X_test = data_model.loc[test_mask, ['X_Driver_Smooth']]
    Y_test = data_model.loc[test_mask, 'Y_Envelope']

    if len(X_train) == 0 or len(X_test) == 0:
        print(f"  错误: 频带 {band_name} 的训练集或测试集为空。")
        return None

    # 7. 拟合一个简单的LGBM: Y_Envelope = f(X_Driver_Smooth)
    #    我们用它来捕捉可能的非线性关系
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 使用一个非常简单的、防止过拟合的LGBM
    lgbm_tester = lgb.LGBMRegressor(
        n_estimators=50,
        num_leaves=5,
        learning_rate=0.05,
        random_state=42,
        n_jobs=1
    )
    
    lgbm_tester.fit(X_train_scaled, Y_train)
    
    # 8. 在测试集上评估 R²
    Y_pred = lgbm_tester.predict(X_test_scaled)
    r2 = r2_score(Y_test, Y_pred)
    
    print(f"  > {band_name} 检验 R²: {r2:.4f}")

    return {
        'model': model_name,
        'band_name': band_name,
        'low_days': freq_band_days[0],
        'high_days': freq_band_days[1],
        'r2_score': r2
    }


# === 主执行脚本 ===
if __name__ == "__main__":
    
    # --- 1. 定义要筛选的频带 (单位: 天) ---
    #    我们筛选短期和中期频率
    BANDS_TO_TEST = {
        # 太阳自转 (宽带)
        'Rotation_Wide': [20, 40],
        # 太阳自转 (核心)
        'Rotation_Core': [25, 30],
        # 里格周期
        'Rieger': [140, 170],
        # 年度周期
        'Annual': [350, 380],
        # QBO (准两年振荡)
        'QBO_1.5_3.0y': [1.5 * 365.25, 3.0 * 365.25],
        # QBO (窄带)
        'QBO_2.2y': [2.0 * 365.25, 2.4 * 365.25]
        # 注意: 22y的Hale周期太长了，不适合做能量包络检验 (它本身就是驱动力尺度)
    }

    # --- 2. 定义模型分割点 ---
    model_splits = {
        'M8+2': '1996-08-01', 
        'M8+3': '1986-08-01',
        'M0+3': '1986-09-01', 
        'M0+2': '1996-08-01'
    }
    
    test_end_date = '2019-11-30'
    
    # --- 3. 加载数据 ---
    try:
        data_source_file = "综合分析结果/汇总拟合1结果.csv"
        print(f"加载源数据: {data_source_file}")
        full_data = pd.read_csv(data_source_file, parse_dates=['日期'])
    except Exception as e:
        print(f"致命错误：无法读取 {data_source_file}。错误: {e}")
        exit()

    
    # --- 4. 开始筛选 ---
    all_results = []
    
    print("\n=== 开始筛选“能量包络调制”的物理频带 ===")
    
    for model_name, split_date in model_splits.items():
        print(f"\n{'='*50}")
        print(f"处理模型: {model_name} (训练截至: {split_date})")
        print(f"{'='*50}")

        # 检查所需列是否存在
        target_col = f'残差_{model_name}'
        driver_col = f'拟合SSN_{model_name}'
        if target_col not in full_data.columns or driver_col not in full_data.columns:
            print(f"警告: 缺少 {target_col} 或 {driver_col}。跳过此模型。")
            continue

        for band_name, freq_band_days in BANDS_TO_TEST.items():
            result = run_modulation_test(
                data_full=full_data,
                model_name=model_name,
                train_end_date=split_date,
                test_end_date=test_end_date,
                band_name=band_name,
                freq_band_days=freq_band_days,
                envelope_smoothing_days=365 # 使用1年平滑窗口
            )
            if result:
                all_results.append(result)

    # --- 5. 总结报告 ---
    if not all_results:
        print("\n所有分析均失败。")
        exit()

    print("\n\n" + "="*60)
    print("      能量包络调制筛选 - 最终总结报告")
    print(" (检验 Y_包络 = f(X_行星拟合) 在测试集上的 R² 分数)")
    print("="*60)

    results_df = pd.DataFrame(all_results)
    
    # 计算每个频带在4个模型上的平均 R²
    summary = results_df.pivot_table(
        index='band_name', 
        columns='model', 
        values='r2_score'
    )
    
    # 计算平均R²并排序
    summary['Average_R2'] = summary.mean(axis=1)
    summary = summary.sort_values(by='Average_R2', ascending=False)
    
    print(summary.to_string(float_format="%.4f"))

    print("\n" + "="*60)
    print("结论:")
    top_band = summary.index[0]
    top_r2 = summary.iloc[0]['Average_R2']
    
    if top_r2 > 0.05:
        print(f"✅ 强烈证据: 频带 '{top_band}' (平均R²={top_r2:.4f}) 的能量包络")
        print("   与11年行星周期有很强的调制关系。")
        print(f"\n👉 建议: 在LGBM全频率梳模型中，应重点关注 {top_band} 频带的调制特征。")
    elif top_r2 > 0.01:
        print(f"✅ 有效证据: 频带 '{top_band}' (平均R²={top_r2:.4f}) 显示了微弱但稳定的调制关系。")
    else:
        print("❌ 证据不足: 所有频带的能量包络与11年周期的相关性 (R²) 都非常低。")
        print("   LGBM中的 'Mod_' 调制特征可能效果有限或依赖于其他机制。")

LightGBM 导入成功。
Scipy 导入成功 (版本: 1.15.2)。
加载源数据: 综合分析结果/汇总拟合1结果.csv

=== 开始筛选“能量包络调制”的物理频带 ===

处理模型: M8+2 (训练截至: 1996-08-01)
  测试频带: Rotation_Wide (20-40 天)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000162 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 53904, number of used features: 1
[LightGBM] [Info] Start training from score 29.679758
  > Rotation_Wide 检验 R²: 0.3943
  测试频带: Rotation_Core (25-30 天)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000167 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 53904, number of used features: 1
[LightGBM] [Info] Start 

# 能量包络筛选，初步拟合结果不佳

In [2]:
import pandas as pd
import numpy as np
import time
import warnings
import os
import joblib

from sklearn.metrics import r2_score
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from skopt import BayesSearchCV
from skopt.space import Real, Integer

try:
    import lightgbm as lgb
    print("LightGBM 导入成功。")
except ImportError:
    print("错误：未找到 lightgbm 库。")
    exit()

try:
    from astropy.time import Time
    import astropy.units as u
    from sunpy.coordinates import sun
except ImportError:
    print("缺少天文学库")
    exit()

warnings.filterwarnings('ignore')

def run_lgbm_optimized_comb_validation( # 重命名函数
    data_full, model_name, train_start_date, train_end_date, test_end_date
):
    """
    运行 LGBM (优化版)
    - 特征: B0 + 所有基础周期
    - 调制: 仅调制 QBO 和 Rotation (基于能量包络筛选)
    - Y变换: QuantileTransformer
    - 调优: 强正则化
    - 功能: 保存模型和全周期预测
    """
    
    data = data_full.copy() 
    target_residual_column = f'残差_{model_name}'
    planetary_fit_column = f'拟合SSN_{model_name}'
    os.makedirs("LGBM_Models_Optimized", exist_ok=True) # 保存到新目录
    
    print(f"\n=== LGBM 优化版 (QBO+Rotation调制): {model_name} ===")
    start_time = time.time()
    
    if target_residual_column not in data.columns or planetary_fit_column not in data.columns:
        print(f"错误: 缺少列 {target_residual_column} 或 {planetary_fit_column}")
        return None, None

    # --- 1. 特征工程：B0 + 全频率梳 (在所有日期上) ---
    print("1. 生成 B0 + 全基础频率梳特征 (全周期)...")
    times_astropy = Time(data['日期'].values)
    
    original_feature_names = []
    
    # B0 (日面中心纬度)
    b0_deg = sun.B0(times_astropy).deg
    data['B0'] = b0_deg
    data['B0_sq'] = b0_deg ** 2
    original_feature_names.extend(['B0', 'B0_sq'])
    
    days_since_start = (data['日期'] - data['日期'].min()).dt.days
    
    # 1. Hale (22y)
    hale_rad = 2 * np.pi * days_since_start / (22.0 * 365.25)
    data['Hale_22y_sin'] = np.sin(hale_rad)
    data['Hale_22y_cos'] = np.cos(hale_rad)
    original_feature_names.extend(['Hale_22y_sin', 'Hale_22y_cos'])

    # 2. QBO (1.5y - 3y)
    qbo_periods = [1.7, 2.2, 2.7] 
    for p in qbo_periods:
        rad = 2 * np.pi * days_since_start / (p * 365.25)
        name = f'QBO_{p:.1f}y'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 3. Annual (1y)
    annual_periods = [360, 365.25, 370] 
    for p in annual_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Annual_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 4. Rieger (154d)
    rieger_rad = 2 * np.pi * days_since_start / 154
    data['Rieger_154d_sin'] = np.sin(rieger_rad)
    data['Rieger_154d_cos'] = np.cos(rieger_rad)
    original_feature_names.extend(['Rieger_154d_sin', 'Rieger_154d_cos'])

    # 5. Rotation (25d-30d)
    rotation_periods = [25, 26, 27, 28, 29, 30] 
    for p in rotation_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Rotation_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])
    
    print(f"基础特征: {len(original_feature_names)} 个")

    # --- 2. 调制特征工程 (重构) ---
    print("2. 生成调制特征 (全周期)...")
    
    train_mask_dates = (data['日期'] >= '1855-12-02') & (data['日期'] <= train_end_date)
    train_mask_clean = train_mask_dates & data[target_residual_column].notna() & data[planetary_fit_column].notna()
    
    if train_mask_clean.sum() == 0:
        print(f"错误：在 {train_start_date} 和 {train_end_date} 之间没有干净的训练数据。")
        return None, None
        
    planetary_fit_values_train = data.loc[train_mask_clean, [planetary_fit_column]]
    scaler_fit = StandardScaler()
    scaler_fit.fit(planetary_fit_values_train)
    
    planetary_fit_values_full = data[[planetary_fit_column]].fillna(0) 
    data['Planetary_Scaled'] = scaler_fit.transform(planetary_fit_values_full).flatten()
    
    # --- 关键修改：基于能量包络筛选结果 ---
    #    只为 Rotation 和 QBO 创建调制特征
    
    # 定义允许被调制的基础特征名称（前缀）
    modulation_allow_list = ['QBO_', 'Rotation_']
    
    features_to_modulate = []
    for feat_name in original_feature_names:
        # 检查特征名是否以 'QBO_' 或 'Rotation_' 开头
        if any(feat_name.startswith(prefix) for prefix in modulation_allow_list):
            features_to_modulate.append(feat_name)

    print(f"筛选出 {len(features_to_modulate)} 个特征进行调制 (基于能量包络检验)。")
    
    # d. 创建调制特征 (仅限允许的列表)
    modulation_feature_names = []
    for feature in features_to_modulate: # <--- 只循环筛选后的子集
        new_feature_name = f'Mod_{feature}'
        data.loc[:, new_feature_name] = data['Planetary_Scaled'] * data[feature]
        modulation_feature_names.append(new_feature_name)
    
    # --- 结束修改 ---
    
    feature_names = original_feature_names + modulation_feature_names
    print(f"总特征数: {len(feature_names)} (30 基础 + {len(modulation_feature_names)} 调制)")
    
    # --- 3. 数据分割与变换 (用于训练) ---
    data_clean = data.dropna(subset=[target_residual_column] + feature_names)
    
    X = data_clean[feature_names]
    Y_raw = data_clean[target_residual_column]
    groups = data_clean['日期'].dt.year
    dates = data_clean['日期']
    
    train_mask = (dates >= train_start_date) & (dates <= train_end_date)
    test_start_date_dt = pd.to_datetime(train_end_date) + pd.Timedelta(days=1)
    test_mask = (dates >= test_start_date_dt) & (dates <= test_end_date)
    
    X_train_raw = X[train_mask]
    Y_train_raw = Y_raw[train_mask]
    X_test_raw = X[test_mask]
    Y_test_raw = Y_raw[test_mask]
    groups_train = groups[train_mask]
    
    if len(X_train_raw) == 0 or len(X_test_raw) == 0:
        print("错误：训练集或测试集为空")
        return None, None

    # a. Fit X 变换
    feature_scaler = StandardScaler()
    X_train = feature_scaler.fit_transform(X_train_raw)
    X_test = feature_scaler.transform(X_test_raw)

    # b. Fit Y 变换
    print("应用 QuantileTransformer 稳定目标变量 Y (残差)...")
    target_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
    Y_train = target_transformer.fit_transform(Y_train_raw.values.reshape(-1, 1)).flatten()
    Y_test = target_transformer.transform(Y_test_raw.values.reshape(-1, 1)).flatten()
    
    print(f"数据: 训练{len(X_train)}, 测试{len(X_test)}")
    
    # --- 4. 贝叶斯优化 - LGBM (强正则化) ---
    n_splits = max(2, min(5, groups_train.nunique()))
    gkf = GroupKFold(n_splits=n_splits)
    
    search_spaces = {
        'n_estimators': Integer(100, 800),
        'num_leaves': Integer(5, 30),
        'learning_rate': Real(1e-3, 0.1, 'log-uniform'),
        'reg_alpha': Real(0.1, 20.0, 'log-uniform'), 
        'reg_lambda': Real(0.1, 20.0, 'log-uniform'),
        'subsample': Real(0.7, 1.0, 'uniform'),
        'colsample_bytree': Real(0.7, 1.0, 'uniform')
    }
    
    lgb_model = lgb.LGBMRegressor(random_state=42, n_jobs=1, 
                                  objective='regression_l1')

    opt_lgbm = BayesSearchCV(
        lgb_model, 
        search_spaces, 
        n_iter=30, 
        cv=gkf, 
        n_jobs=-1,
        random_state=42, 
        scoring='r2'
    )
    
    print("训练LGBM (优化版)...")
    opt_lgbm.fit(X_train, Y_train, groups=groups_train)
    
    # --- 5. 结果分析 ---
    best_model = opt_lgbm.best_estimator_
    Y_pred_test = best_model.predict(X_test)
    final_r2 = r2_score(Y_test, Y_pred_test) 
    
    print(f"\nLGBM 优化版 R² (在变换后的Y上): {final_r2:.4f}")
    if final_r2 > 0:
        print("✅ 成功！模型R²为正值。")
    else:
        print("❌ R²仍为负值。")
        
    print(f"最佳参数: {opt_lgbm.best_params_}")
    
    importances = pd.Series(best_model.feature_importances_, index=feature_names)
    print("\n特征重要性 (Top 20):")
    top_features = importances.sort_values(ascending=False).head(20) 
    print(top_features.to_string())
    
    mod_features_in_top = [f for f in top_features.index if f.startswith('Mod_')]
    print(f"\nTop 20 中，有 {len(mod_features_in_top)} 个是调制特征。")
    
    # --- 6. 新增功能：保存模型 ---
    print("\n保存模型及 Scalers...")
    model_path = f"LGBM_Models_Optimized/{model_name}_best_lgbm.joblib"
    fscaler_path = f"LGBM_Models_Optimized/{model_name}_feature_scaler.joblib"
    tscaler_path = f"LGBM_Models_Optimized/{model_name}_target_transformer.joblib"
    
    joblib.dump(best_model, model_path)
    joblib.dump(feature_scaler, fscaler_path)
    joblib.dump(target_transformer, tscaler_path)
    print(f"模型已保存: {model_path}")

    # --- 7. 新增功能：生成全周期预测 ---
    print("生成全周期（至2050年）预测...")
    X_full = data[feature_names].fillna(0) 
    
    X_full_scaled = feature_scaler.transform(X_full)
    Y_pred_full_transformed = best_model.predict(X_full_scaled)
    
    Y_pred_full_raw = target_transformer.inverse_transform(Y_pred_full_transformed.reshape(-1, 1)).flatten()
    
    pred_col_name = f'LGBM_Pred_Opt_{model_name}' # 优化版列名
    resid_col_name = f'LGBM_Resid_Opt_{model_name}' # 优化版列名
    
    data[pred_col_name] = Y_pred_full_raw
    data[resid_col_name] = data[target_residual_column] - data[pred_col_name]
    
    stats_dict = {
        'model': model_name,
        'r2_score_transformed': final_r2,
        'n_features': len(feature_names),
        'n_mod_features': len(modulation_feature_names),
        'top_20_mod_count': len(mod_features_in_top),
        'best_params': opt_lgbm.best_params_,
        'training_time': time.time() - start_time
    }

    return data[['日期', pred_col_name, resid_col_name]], stats_dict


# 主执行
if __name__ == "__main__":
    model_splits = {
        'M8+2': '1996-08-01', 
        'M8+3': '1986-08-01',
        'M0+3': '1986-09-01', 
        'M0+2': '1996-08-01'
    }
    
    print("=== 运行 LGBM (物理筛选优化版) ===")
    print("策略: 基础全频率梳 + (QBO+Rotation)调制 + 目标稳定 + 强正则化")
    
    try:
        data_source_file = "综合分析结果/汇总拟合1结果.csv"
        print(f"\n加载源数据: {data_source_file}")
        full_data_to_save = pd.read_csv(data_source_file, parse_dates=['日期'])
    except Exception as e:
        print(f"致命错误：无法读取 {data_source_file}。错误: {e}")
        exit()

    max_date_in_file = full_data_to_save['日期'].max()
    target_date = pd.to_datetime('2050-12-31')
    
    if max_date_in_file < target_date:
        print(f"警告: 源数据最大日期为 {max_date_in_file}。")
        print(f"警告: 正在扩展日期范围至 {target_date}。")
        all_dates_df = pd.DataFrame({'日期': pd.date_range(
            start=full_data_to_save['日期'].min(), 
            end=target_date, 
            freq='D'
        )})
        full_data_to_save = pd.merge(all_dates_df, full_data_to_save, on='日期', how='left')

    results_stats_list = []
    
    for model_name, split_date in model_splits.items():
        print(f"\n{'='*50}")
        print(f"处理模型: {model_name} (训练结束于 {split_date})")
        print(f"{'='*50}")
        
        result_df, stats_dict = run_lgbm_optimized_comb_validation(
            data_full=full_data_to_save.copy(), 
            model_name=model_name,
            train_start_date='1855-12-02',
            train_end_date=split_date,
            test_end_date='2019-11-30'
        )
        
        if result_df is not None and stats_dict is not None:
            results_stats_list.append(stats_dict)
            full_data_to_save = pd.merge(full_data_to_save, result_df, on='日期', how='left')
    
    if results_stats_list:
        output_filename = "综合分析结果/汇总拟合2_LGBM结果_Optimized.csv" # 新文件名
        print(f"\n{'='*60}")
        print(f"保存所有模型的优化版二次拟合和残差到: {output_filename}")
        try:
            full_data_to_save.to_csv(output_filename, index=False, float_format='%.6f')
            print(f"保存成功。文件包含 {len(full_data_to_save)} 行数据。")
        except Exception as e:
            print(f"保存CSV失败: {e}")
        
        results_to_print = []
        for r in results_stats_list:
            results_to_print.append({
                'model': r['model'],
                'r2_score': r['r2_score_transformed'],
                'n_feat_total': r['n_features'],
                'n_feat_mod': r['n_mod_features'],
                'top_20_mod': r['top_20_mod_count'],
                'time(s)': r['training_time']
            })
            
        results_df = pd.DataFrame(results_to_print)
        print("\n" + "="*60)
        print("LGBM 物理筛选优化版 总结")
        print("="*60)
        print(results_df.to_string(index=False, float_format="%.4f"))
        
        avg_r2 = results_df['r2_score'].mean()
        print(f"\n平均R² (在变换后的Y上): {avg_r2:.4f}")
        
        # 与之前的0.1265 (或 0.1198) 比较
        if avg_r2 > 0.13:
            print(f"✅✅ 巨大成功: 优化版R² ({avg_r2:.4f}) 高于基线 (约 0.12)！")
        elif avg_r2 > 0.11:
            print(f"✅ 成功: 优化版R² ({avg_r2:.4f}) 与基线 (约 0.12) 相当。")
            print("   这证明了模型的鲁棒性，且物理意义更强。")
        else:
            print(f"❌ 注意: 优化版R² ({avg_r2:.4f}) 低于基线 (约 0.12)。")
            print("   这可能意味着被删除的特征中仍有微弱信号。")

    else:
        print("\n所有模型处理失败")

LightGBM 导入成功。
=== 运行 LGBM (物理筛选优化版) ===
策略: 基础全频率梳 + (QBO+Rotation)调制 + 目标稳定 + 强正则化

加载源数据: 综合分析结果/汇总拟合1结果.csv

处理模型: M8+2 (训练结束于 1996-08-01)

=== LGBM 优化版 (QBO+Rotation调制): M8+2 ===
1. 生成 B0 + 全基础频率梳特征 (全周期)...
基础特征: 30 个
2. 生成调制特征 (全周期)...
筛选出 18 个特征进行调制 (基于能量包络检验)。
总特征数: 48 (30 基础 + 18 调制)
应用 QuantileTransformer 稳定目标变量 Y (残差)...
数据: 训练51378, 测试8521
训练LGBM (优化版)...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002372 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 12238
[LightGBM] [Info] Number of data points in the train set: 51378, number of used features: 48
[LightGBM] [Info] Start training from score 0.017457

LGBM 优化版 R² (在变换后的Y上): -0.0935
❌ R²仍为负值。
最佳参数: OrderedDict({'colsample_bytree': 0.7, 'learning_rate': 0.0015805316449195594, 'n_estimators': 800, 'num_leaves': 19, 'reg_alpha': 0.13339802460402347, 'reg_lambda': 0.1, 'sub

# 补充行星拟合可能遗漏的长期信息

In [2]:
import pandas as pd
import numpy as np
import time
import warnings
import os  # 新增：用于创建目录
import joblib  # 新增：用于保存模型

from sklearn.metrics import r2_score
from sklearn.model_selection import GroupKFold
# 导入 QuantileTransformer (关键修复)
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# --- 核心：非线性模型导入 ---
try:
    import lightgbm as lgb
    print("LightGBM 导入成功。")
except ImportError:
    print("错误：未找到 lightgbm 库。")
    print("请安装: pip install lightgbm")
    exit()

# 天文学库导入
try:
    from astropy.time import Time
    import astropy.units as u
    from sunpy.coordinates import sun
except ImportError:
    print("缺少天文学库")
    exit()

warnings.filterwarnings('ignore')

def run_lgbm_full_comb_validation(
    data_full, model_name, train_start_date, train_end_date, test_end_date
):
    """
    运行 LGBM (最终全频率梳版)
    - 特征: B0 + 所有著名周期 (22y, QBO, 1y, 154d, 25-30d)
    - 调制: 11年行星拟合
    - Y变换: QuantileTransformer
    - 调优: 强正则化
    - 新增: 保存模型和全周期预测
    """
    
    # 使用传入的完整数据
    data = data_full.copy() 
    
    target_residual_column = f'残差_{model_name}'
    planetary_fit_column = f'拟合SSN_{model_name}'
    
    # 确保模型保存目录存在
    os.makedirs("LGBM_Models", exist_ok=True)
    
    print(f"\n=== LGBM 最终全频率梳版: {model_name} ===")
    start_time = time.time()
    
    # 检查必要的列是否存在
    if target_residual_column not in data.columns or planetary_fit_column not in data.columns:
        print(f"错误: 缺少列 {target_residual_column} 或 {planetary_fit_column}")
        return None, None

    # --- 1. 特征工程：B0 + 全频率梳 (在所有日期上) ---
    print("1. 生成 B0 星历特征 + 全频率梳特征 (全周期)...")
    times_astropy = Time(data['日期'].values)
    
    original_feature_names = []
    
    # B0 (日面中心纬度) - 真实年度振幅
    b0_deg = sun.B0(times_astropy).deg
    data['B0'] = b0_deg
    data['B0_sq'] = b0_deg ** 2
    original_feature_names.extend(['B0', 'B0_sq'])
    
    # 基于时间的固定频率特征
    days_since_start = (data['日期'] - data['日期'].min()).dt.days
    
    # === 定义我们的频率梳 ===
    
    # 1. Hale (22y)
    hale_rad = 2 * np.pi * days_since_start / (22.0 * 365.25)
    data['Hale_22y_sin'] = np.sin(hale_rad)
    data['Hale_22y_cos'] = np.cos(hale_rad)
    original_feature_names.extend(['Hale_22y_sin', 'Hale_22y_cos'])

    # 2. QBO (1.5y - 3y)
    qbo_periods = [1.7, 2.2, 2.7] # 年
    for p in qbo_periods:
        rad = 2 * np.pi * days_since_start / (p * 365.25)
        name = f'QBO_{p:.1f}y'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 3. Annual (1y)
    annual_periods = [360, 365.25, 370] # 天
    for p in annual_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Annual_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 4. Rieger (154d)
    rieger_rad = 2 * np.pi * days_since_start / 154
    data['Rieger_154d_sin'] = np.sin(rieger_rad)
    data['Rieger_154d_cos'] = np.cos(rieger_rad)
    original_feature_names.extend(['Rieger_154d_sin', 'Rieger_154d_cos'])

    # 5. Rotation (25d-30d)
    rotation_periods = [25, 26, 27, 28, 29, 30] # 天
    for p in rotation_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Rotation_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])
    
    
    # --- 2. 调制特征工程 (重构) ---
    # 目标：在训练集上 fit Scaler，在全周期上 transform
    print("2. 生成调制特征 (全周期)...")
    
    # a. 创建用于拟合 Scaler 的训练掩码
    # 训练集掩码（仅用于拟合 Scaler 和模型）
    train_mask_dates = (data['日期'] >= train_start_date) & (data['日期'] <= train_end_date)
    # 必须是干净的、非NaN的数据
    train_mask_clean = train_mask_dates & data[target_residual_column].notna() & data[planetary_fit_column].notna()
    
    if train_mask_clean.sum() == 0:
        print(f"错误：在 {train_start_date} 和 {train_end_date} 之间没有干净的训练数据。")
        return None, None
        
    # b. Fit 调制器 (11年周期信号) Scaler (仅在训练数据上)
    planetary_fit_values_train = data.loc[train_mask_clean, [planetary_fit_column]]
    scaler_fit = StandardScaler()
    scaler_fit.fit(planetary_fit_values_train)
    
    # c. Transform (全周期)
    # 警告：如果 planetary_fit_column 在未来是 NaN，这里会产生 NaN
    planetary_fit_values_full = data[[planetary_fit_column]].fillna(0) # 假设未来拟合为0（如果丢失），这不理想但能防止崩溃
    data['Planetary_Scaled'] = scaler_fit.transform(planetary_fit_values_full).flatten()

    # --- [ *** 新增代码 *** ] ---
    # 2.c.1: 创建变化率特征 (1阶差分)
    # 这代表了11年周期的“上升相”（正值）或“下降相”（负值）
    print("... 正在添加 'Planetary_Scaled_Diff' (变化率) 特征...")
    data['Planetary_Scaled_Diff'] = data['Planetary_Scaled'].diff(1).fillna(0)
    
    # 2.c.2: 将其添加到 *基础* 特征列表
    # 关键：这样它在下一步中会自动获得自己的 "Mod_" 版本
    original_feature_names.append('Planetary_Scaled_Diff')
    # --- [ *** 新增代码结束 *** ] ---
    
    # d. 创建调制特征 (全周期)
    modulation_feature_names = []
    for feature in original_feature_names:
        new_feature_name = f'Mod_{feature}'
        data.loc[:, new_feature_name] = data['Planetary_Scaled'] * data[feature]
        modulation_feature_names.append(new_feature_name)
    
    feature_names = original_feature_names + modulation_feature_names
    print(f"总特征数: {len(feature_names)} (全频率梳 + 变化率)")
    
    # --- 3. 数据分割与变换 (用于训练) ---
    
    # 仅使用具有有效残差的行进行训练和测试
    data_clean = data.dropna(subset=[target_residual_column] + feature_names)
    
    X = data_clean[feature_names]
    Y_raw = data_clean[target_residual_column] # Y 原始值
    groups = data_clean['日期'].dt.year
    dates = data_clean['日期']
    
    train_mask = (dates >= train_start_date) & (dates <= train_end_date)
    test_start_date_dt = pd.to_datetime(train_end_date) + pd.Timedelta(days=1)
    test_mask = (dates >= test_start_date_dt) & (dates <= test_end_date)
    
    X_train_raw = X[train_mask]
    Y_train_raw = Y_raw[train_mask]
    X_test_raw = X[test_mask]
    Y_test_raw = Y_raw[test_mask]
    groups_train = groups[train_mask]
    
    if len(X_train_raw) == 0 or len(X_test_raw) == 0:
        print("错误：训练集或测试集为空")
        return None, None

    # a. Fit X 变换 (StandardScaler)
    feature_scaler = StandardScaler()
    X_train = feature_scaler.fit_transform(X_train_raw)
    X_test = feature_scaler.transform(X_test_raw)

    # b. Fit Y 变换 (QuantileTransformer) - 关键修复
    print("应用 QuantileTransformer 稳定目标变量 Y (残差)...")
    target_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
    Y_train = target_transformer.fit_transform(Y_train_raw.values.reshape(-1, 1)).flatten()
    Y_test = target_transformer.transform(Y_test_raw.values.reshape(-1, 1)).flatten()
    
    print(f"数据: 训练{len(X_train)}, 测试{len(X_test)}")
    
    # --- 4. 贝叶斯优化 - LGBM (强正则化) ---
    n_splits = max(2, min(5, groups_train.nunique()))
    gkf = GroupKFold(n_splits=n_splits)
    
    search_spaces = {
        'n_estimators': Integer(100, 800),
        'num_leaves': Integer(5, 30),
        'learning_rate': Real(1e-3, 0.1, 'log-uniform'),
        'reg_alpha': Real(0.1, 20.0, 'log-uniform'), 
        'reg_lambda': Real(0.1, 20.0, 'log-uniform'),
        'subsample': Real(0.7, 1.0, 'uniform'),
        'colsample_bytree': Real(0.7, 1.0, 'uniform')
    }
    
    lgb_model = lgb.LGBMRegressor(
        random_state=42, 
        n_jobs=1, 
        objective='regression_l1',
        force_row_wise=True, # 强制使用行优先，消除自动检测的提示
        verbose=-1           # 抑制所有 LightGBM 的内部输出（推荐）
    )

    opt_lgbm = BayesSearchCV(
        lgb_model, 
        search_spaces, 
        n_iter=100, # <-- 从 30 增加到 50 
        cv=gkf, 
        n_jobs=-2,
        random_state=42, 
        scoring='r2'
    )
    
    print("训练LGBM (全频率梳版)...")
    opt_lgbm.fit(X_train, Y_train, groups=groups_train)
    
    # --- 5. 结果分析 ---
    best_model = opt_lgbm.best_estimator_
    Y_pred_test = best_model.predict(X_test)
    final_r2 = r2_score(Y_test, Y_pred_test) 
    
    print(f"\nLGBM 全频率梳版 R² (在变换后的Y上): {final_r2:.4f}")
    if final_r2 > 0:
        print("✅ 成功！模型R²为正值。")
    else:
        print("❌ R²仍为负值。")
        
    print(f"最佳参数: {opt_lgbm.best_params_}")
    
    # 特征重要性
    importances = pd.Series(best_model.feature_importances_, index=feature_names)
    print("\n特征重要性 (Top 20):")
    top_features = importances.sort_values(ascending=False).head(20) 
    print(top_features.to_string())
    
    mod_features_in_top = [f for f in top_features.index if f.startswith('Mod_')]
    print(f"\nTop 20 中，有 {len(mod_features_in_top)} 个是调制特征。")
    
    # --- 6. 新增功能：保存模型 ---
    print("\n保存模型及 Scalers...")
    model_path = f"LGBM_Models/{model_name}_best_lgbm.joblib"
    fscaler_path = f"LGBM_Models/{model_name}_feature_scaler.joblib"
    tscaler_path = f"LGBM_Models/{model_name}_target_transformer.joblib"
    
    joblib.dump(best_model, model_path)
    joblib.dump(feature_scaler, fscaler_path)
    joblib.dump(target_transformer, tscaler_path)
    print(f"模型已保存: {model_path}")

    # --- 7. 新增功能：生成全周期预测 ---
    print("生成全周期（至2050年）预测...")
    
    # 准备全周期的 X 特征
    # 必须填充特征中的NaN（例如，如果 Planetary_Scaled 最终为NaN）
    X_full = data[feature_names].fillna(0) 
    
    X_full_scaled = feature_scaler.transform(X_full)
    Y_pred_full_transformed = best_model.predict(X_full_scaled)
    
    # 反变换回原始尺度
    Y_pred_full_raw = target_transformer.inverse_transform(Y_pred_full_transformed.reshape(-1, 1)).flatten()
    
    # 创建预测列和二次残差列
    pred_col_name = f'LGBM_Pred_{model_name}'
    resid_col_name = f'LGBM_Resid_{model_name}'
    
    data[pred_col_name] = Y_pred_full_raw
    # 原始残差 - LGBM预测的残差 = 二次残差
    data[resid_col_name] = data[target_residual_column] - data[pred_col_name]
    
    # 准备统计字典
    stats_dict = {
        'model': model_name,
        'r2_score_transformed': final_r2,
        'n_features': len(feature_names),
        'top_20_mod_count': len(mod_features_in_top),
        'best_params': opt_lgbm.best_params_,
        'training_time': time.time() - start_time
    }

    # 返回包含新列的DataFrame（仅含必要列）和统计结果
    return data[['日期', pred_col_name, resid_col_name]], stats_dict


# 主执行 - 运行 LGBM 全频率梳版
if __name__ == "__main__":
    model_splits = {
        'M8+2': '1996-08-01', 
        'M8+3': '1986-08-01',
        'M0+3': '1986-09-01', 
        'M0+2': '1996-08-01'
    }
    
    print("=== 运行 LGBM (全频率梳版: B0 + 22y, QBO, 1y, 154d, 25-30d) ===")
    print("策略: 物理全频率梳 + 11年调制 + 目标稳定 + 强正则化")
    print("新增: 保存模型和全周期二次残差")
    
    # --- 新增：在循环外加载和准备数据 ---
    try:
        data_source_file = "综合分析结果/汇总拟合1结果.csv"
        print(f"\n加载源数据: {data_source_file}")
        full_data_to_save = pd.read_csv(data_source_file, parse_dates=['日期'])
    except Exception as e:
        print(f"致命错误：无法读取 {data_source_file}。错误: {e}")
        exit()

    # 检查并扩展日期至2050年
    max_date_in_file = full_data_to_save['日期'].max()
    target_date = pd.to_datetime('2050-12-31')
    
    if max_date_in_file < target_date:
        print(f"警告: 源数据最大日期为 {max_date_in_file}。")
        print(f"警告: 正在扩展日期范围至 {target_date}。")
        print("警告: 请确保 '拟合SSN' (行星拟合) 列在源文件中已外推至2050年，否则预测将不准确！")
        
        all_dates_df = pd.DataFrame({'日期': pd.date_range(
            start=full_data_to_save['日期'].min(), 
            end=target_date, 
            freq='D'
        )})
        full_data_to_save = pd.merge(all_dates_df, full_data_to_save, on='日期', how='left')

    # 用于保存统计结果
    results_stats_list = []
    
    # 循环处理每个模型
    for model_name, split_date in model_splits.items():
        print(f"\n{'='*50}")
        print(f"处理模型: {model_name} (训练结束于 {split_date})")
        print(f"{'='*50}")
        
        # 传入完整数据
        result_df, stats_dict = run_lgbm_full_comb_validation(
            data_full=full_data_to_save.copy(), # 传入数据的副本
            model_name=model_name,
            train_start_date='1855-12-02',
            train_end_date=split_date,
            test_end_date='2019-11-30'
        )
        
        if result_df is not None and stats_dict is not None:
            results_stats_list.append(stats_dict)
            # 将该模型的结果合并回主DataFrame
            full_data_to_save = pd.merge(full_data_to_save, result_df, on='日期', how='left')
    
    # --- 新增：保存包含所有结果的CSV ---
    if results_stats_list:
        output_filename = "综合分析结果/汇总拟合2_LGBM结果.csv"
        print(f"\n{'='*60}")
        print(f"保存所有模型的二次拟合和残差到: {output_filename}")
        try:
            full_data_to_save.to_csv(output_filename, index=False, float_format='%.6f')
            print(f"保存成功。文件包含 {len(full_data_to_save)} 行数据。")
        except Exception as e:
            print(f"保存CSV失败: {e}")
        
        # --- 打印总结报告 (使用统计数据) ---
        results_to_print = []
        for r in results_stats_list:
            results_to_print.append({
                'model': r['model'],
                'r2_score': r['r2_score_transformed'],
                'n_features': r['n_features'],
                'top_20_mod': r['top_20_mod_count'],
                'time(s)': r['training_time']
            })
            
        results_df = pd.DataFrame(results_to_print)
        print("\n" + "="*60)
        print("LGBM 全频率梳版 总结")
        print("="*60)
        print(results_df.to_string(index=False, float_format="%.4f"))
        
        avg_r2 = results_df['r2_score'].mean()
        print(f"\n平均R² (在变换后的Y上): {avg_r2:.4f}")
        
        positive_r2 = results_df[results_df['r2_score'] > 0]
        if len(positive_r2) == 4:
            print("✅✅✅ 巨大成功: 所有 4 个模型 R² 均为正值！")
        elif len(positive_r2) > 0:
            print(f"✅ 成功: {len(positive_r2)} 个模型获得正R²。")
        else:
            print("❌ 失败: 所有模型R²仍为负值。")
        
        if results_df['top_20_mod'].mean() >= 10: 
             print("💡 洞察: 调制特征 (Mod_) 在模型中占有重要地位。")

    else:
        print("\n所有模型处理失败")

LightGBM 导入成功。
=== 运行 LGBM (全频率梳版: B0 + 22y, QBO, 1y, 154d, 25-30d) ===
策略: 物理全频率梳 + 11年调制 + 目标稳定 + 强正则化
新增: 保存模型和全周期二次残差

加载源数据: 综合分析结果/汇总拟合1结果.csv

处理模型: M8+2 (训练结束于 1996-08-01)

=== LGBM 最终全频率梳版: M8+2 ===
1. 生成 B0 星历特征 + 全频率梳特征 (全周期)...
2. 生成调制特征 (全周期)...
... 正在添加 'Planetary_Scaled_Diff' (变化率) 特征...
总特征数: 62 (全频率梳 + 变化率)
应用 QuantileTransformer 稳定目标变量 Y (残差)...
数据: 训练51378, 测试8521
训练LGBM (全频率梳版)...

LGBM 全频率梳版 R² (在变换后的Y上): 0.0105
✅ 成功！模型R²为正值。
最佳参数: OrderedDict({'colsample_bytree': 1.0, 'learning_rate': 0.016395645320842304, 'n_estimators': 100, 'num_leaves': 14, 'reg_alpha': 3.6710523504410557, 'reg_lambda': 0.1, 'subsample': 1.0})

特征重要性 (Top 20):
Mod_Hale_22y_sin        317
Hale_22y_cos            233
Hale_22y_sin            162
Mod_Hale_22y_cos        154
Mod_QBO_2.7y_cos        105
Mod_QBO_2.7y_sin         42
Mod_QBO_2.2y_sin         40
QBO_2.7y_cos             29
Mod_Annual_370d_cos      29
Mod_QBO_2.2y_cos         24
Mod_Annual_365d_sin      19
Mod_QBO_1.7y_cos         17
Mod

# 在11变化率的基础上，增加“加速度”。最优。

In [3]:
import pandas as pd
import numpy as np
import time
import warnings
import os  # 新增：用于创建目录
import joblib  # 新增：用于保存模型

from sklearn.metrics import r2_score
from sklearn.model_selection import GroupKFold
# 导入 QuantileTransformer (关键修复)
from sklearn.preprocessing import StandardScaler, QuantileTransformer
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# --- 核心：非线性模型导入 ---
try:
    import lightgbm as lgb
    print("LightGBM 导入成功。")
except ImportError:
    print("错误：未找到 lightgbm 库。")
    print("请安装: pip install lightgbm")
    exit()

# 天文学库导入
try:
    from astropy.time import Time
    import astropy.units as u
    from sunpy.coordinates import sun
except ImportError:
    print("缺少天文学库")
    exit()

warnings.filterwarnings('ignore')

def run_lgbm_full_comb_validation(
    data_full, model_name, train_start_date, train_end_date, test_end_date
):
    """
    运行 LGBM (最终全频率梳版)
    - 特征: B0 + 所有著名周期 (22y, QBO, 1y, 154d, 25-30d)
    - 调制: 11年行星拟合
    - Y变换: QuantileTransformer
    - 调优: 强正则化
    - 新增: 保存模型和全周期预测
    """
    
    # 使用传入的完整数据
    data = data_full.copy() 
    
    target_residual_column = f'残差_{model_name}'
    planetary_fit_column = f'拟合SSN_{model_name}'
    
    # 确保模型保存目录存在
    os.makedirs("LGBM_Models", exist_ok=True)
    
    print(f"\n=== LGBM 最终全频率梳版: {model_name} ===")
    start_time = time.time()
    
    # 检查必要的列是否存在
    if target_residual_column not in data.columns or planetary_fit_column not in data.columns:
        print(f"错误: 缺少列 {target_residual_column} 或 {planetary_fit_column}")
        return None, None

    # --- 1. 特征工程：B0 + 全频率梳 (在所有日期上) ---
    print("1. 生成 B0 星历特征 + 全频率梳特征 (全周期)...")
    times_astropy = Time(data['日期'].values)
    
    original_feature_names = []
    
    # B0 (日面中心纬度) - 真实年度振幅
    b0_deg = sun.B0(times_astropy).deg
    data['B0'] = b0_deg
    data['B0_sq'] = b0_deg ** 2
    original_feature_names.extend(['B0', 'B0_sq'])
    
    # 基于时间的固定频率特征
    days_since_start = (data['日期'] - data['日期'].min()).dt.days
    
    # === 定义我们的频率梳 ===
    
    # 1. Hale (22y)
    hale_rad = 2 * np.pi * days_since_start / (22.0 * 365.25)
    data['Hale_22y_sin'] = np.sin(hale_rad)
    data['Hale_22y_cos'] = np.cos(hale_rad)
    original_feature_names.extend(['Hale_22y_sin', 'Hale_22y_cos'])

    # 2. QBO (1.5y - 3y)
    qbo_periods = [1.7, 2.2, 2.7] # 年
    for p in qbo_periods:
        rad = 2 * np.pi * days_since_start / (p * 365.25)
        name = f'QBO_{p:.1f}y'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 3. Annual (1y)
    annual_periods = [360, 365.25, 370] # 天
    for p in annual_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Annual_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])

    # 4. Rieger (154d)
    rieger_rad = 2 * np.pi * days_since_start / 154
    data['Rieger_154d_sin'] = np.sin(rieger_rad)
    data['Rieger_154d_cos'] = np.cos(rieger_rad)
    original_feature_names.extend(['Rieger_154d_sin', 'Rieger_154d_cos'])

    # 5. Rotation (25d-30d)
    rotation_periods = [25, 26, 27, 28, 29, 30] # 天
    for p in rotation_periods:
        rad = 2 * np.pi * days_since_start / p
        name = f'Rotation_{p:.0f}d'
        data[f'{name}_sin'] = np.sin(rad)
        data[f'{name}_cos'] = np.cos(rad)
        original_feature_names.extend([f'{name}_sin', f'{name}_cos'])
    
    
    # --- 2. 调制特征工程 (重构) ---
    # 目标：在训练集上 fit Scaler，在全周期上 transform
    print("2. 生成调制特征 (全周期)...")
    
    # a. 创建用于拟合 Scaler 的训练掩码
    # 训练集掩码（仅用于拟合 Scaler 和模型）
    train_mask_dates = (data['日期'] >= train_start_date) & (data['日期'] <= train_end_date)
    # 必须是干净的、非NaN的数据
    train_mask_clean = train_mask_dates & data[target_residual_column].notna() & data[planetary_fit_column].notna()
    
    if train_mask_clean.sum() == 0:
        print(f"错误：在 {train_start_date} 和 {train_end_date} 之间没有干净的训练数据。")
        return None, None
        
    # b. Fit 调制器 (11年周期信号) Scaler (仅在训练数据上)
    planetary_fit_values_train = data.loc[train_mask_clean, [planetary_fit_column]]
    scaler_fit = StandardScaler()
    scaler_fit.fit(planetary_fit_values_train)
    
    # c. Transform (全周期)
    # 警告：如果 planetary_fit_column 在未来是 NaN，这里会产生 NaN
    planetary_fit_values_full = data[[planetary_fit_column]].fillna(0) # 假设未来拟合为0（如果丢失），这不理想但能防止崩溃
    data['Planetary_Scaled'] = scaler_fit.transform(planetary_fit_values_full).flatten()

    # --- [ *** 此处修改 *** ] ---
    
    # 2.c.1: 创建变化率特征 (速度)
    print("... 正在添加 'Planetary_Scaled_Diff' (速度) 特征...")
    data['Planetary_Scaled_Diff'] = data['Planetary_Scaled'].diff(1).fillna(0)
    
    # 2.c.2: 创建加速度特征 (速度的变化率)
    print("... 正在添加 'Planetary_Scaled_Accel' (加速度) 特征...")
    data['Planetary_Scaled_Accel'] = data['Planetary_Scaled_Diff'].diff(1).fillna(0)

    # 2.c.3: 将它们都添加到 *基础* 特征列表
    # 关键：这样它们在下一步中会自动获得自己的 "Mod_" 版本
    original_feature_names.append('Planetary_Scaled_Diff')
    original_feature_names.append('Planetary_Scaled_Accel')
    # --- [ *** 修改结束 *** ] ---
    
    # d. 创建调制特征 (全周期)
    modulation_feature_names = []
    for feature in original_feature_names:
        new_feature_name = f'Mod_{feature}'
        data.loc[:, new_feature_name] = data['Planetary_Scaled'] * data[feature]
        modulation_feature_names.append(new_feature_name)
    
    feature_names = original_feature_names + modulation_feature_names

    # 打印语句现在会显示 32 个基础特征 (30个原始 + 1个Diff + 1个Accel)
    print(f"基础特征数量 (包括 Diff 和 Accel): {len(original_feature_names)}") 
    # 你的总特征数现在应该是 64 (32 + 32)
    print(f"总特征数: {len(feature_names)} (全频率梳 + 速度 + 加速度)")
    
    # --- 3. 数据分割与变换 (用于训练) ---
    
    # 仅使用具有有效残差的行进行训练和测试
    data_clean = data.dropna(subset=[target_residual_column] + feature_names)
    
    X = data_clean[feature_names]
    Y_raw = data_clean[target_residual_column] # Y 原始值
    groups = data_clean['日期'].dt.year
    dates = data_clean['日期']
    
    train_mask = (dates >= train_start_date) & (dates <= train_end_date)
    test_start_date_dt = pd.to_datetime(train_end_date) + pd.Timedelta(days=1)
    test_mask = (dates >= test_start_date_dt) & (dates <= test_end_date)
    
    X_train_raw = X[train_mask]
    Y_train_raw = Y_raw[train_mask]
    X_test_raw = X[test_mask]
    Y_test_raw = Y_raw[test_mask]
    groups_train = groups[train_mask]
    
    if len(X_train_raw) == 0 or len(X_test_raw) == 0:
        print("错误：训练集或测试集为空")
        return None, None

    # a. Fit X 变换 (StandardScaler)
    feature_scaler = StandardScaler()
    X_train = feature_scaler.fit_transform(X_train_raw)
    X_test = feature_scaler.transform(X_test_raw)

    # b. Fit Y 变换 (QuantileTransformer) - 关键修复
    print("应用 QuantileTransformer 稳定目标变量 Y (残差)...")
    target_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
    Y_train = target_transformer.fit_transform(Y_train_raw.values.reshape(-1, 1)).flatten()
    Y_test = target_transformer.transform(Y_test_raw.values.reshape(-1, 1)).flatten()
    
    print(f"数据: 训练{len(X_train)}, 测试{len(X_test)}")
    
    # --- 4. 贝叶斯优化 - LGBM (强正则化) ---
    n_splits = max(2, min(5, groups_train.nunique()))
    gkf = GroupKFold(n_splits=n_splits)
    
    search_spaces = {
        'n_estimators': Integer(100, 800),
        'num_leaves': Integer(5, 30),
        'learning_rate': Real(1e-3, 0.1, 'log-uniform'),
        'reg_alpha': Real(0.1, 20.0, 'log-uniform'), 
        'reg_lambda': Real(0.1, 20.0, 'log-uniform'),
        'subsample': Real(0.7, 1.0, 'uniform'),
        'colsample_bytree': Real(0.7, 1.0, 'uniform')
    }
    
    lgb_model = lgb.LGBMRegressor(
        random_state=42, 
        n_jobs=1, 
        objective='regression_l1',
        force_row_wise=True, # 强制使用行优先，消除自动检测的提示
        verbose=-1           # 抑制所有 LightGBM 的内部输出（推荐）
    )

    opt_lgbm = BayesSearchCV(
        lgb_model, 
        search_spaces, 
        n_iter=100, 
        cv=gkf, 
        n_jobs=-1,
        random_state=42, 
        scoring='r2'
    )
    
    print("训练LGBM (全频率梳版)...")
    opt_lgbm.fit(X_train, Y_train, groups=groups_train)
    
    # --- 5. 结果分析 ---
    best_model = opt_lgbm.best_estimator_
    
    # 预测检验集结果
    Y_pred_test = best_model.predict(X_test)
    
    # 计算 M_T_R2 (在变换后的Y上)
    final_r2_test = r2_score(Y_test, Y_pred_test) 
    
    # 获取 M_CV_R2 (在变换后的Y上)
    best_cv_score = opt_lgbm.best_score_
    
    print(f"\nLGBM 最佳 CV R² (M_CV_R2): {best_cv_score:.4f}")
    print(f"LGBM 检验集 Test R² (M_T_R2): {final_r2_test:.4f}")
    
    if final_r2_test > 0: # 使用修正后的变量名进行检查
        print("✅ 成功！模型R²为正值。")
    else:
        print("❌ R²仍为负值。")
        
    print(f"最佳参数: {opt_lgbm.best_params_}")
    
    # 特征重要性
    importances = pd.Series(best_model.feature_importances_, index=feature_names)
    print("\n特征重要性 (Top 20):")
    top_features = importances.sort_values(ascending=False).head(20)    
    print(top_features.to_string())
    
    mod_features_in_top = [f for f in top_features.index if f.startswith('Mod_')]
    print(f"\nTop 20 中，有 {len(mod_features_in_top)} 个是调制特征。")
    
    # --- 6. 新增功能：保存模型 ---
    print("\n保存模型及 Scalers...")
    model_path = f"LGBM_Models/{model_name}_best_lgbm.joblib"
    fscaler_path = f"LGBM_Models/{model_name}_feature_scaler.joblib"
    tscaler_path = f"LGBM_Models/{model_name}_target_transformer.joblib"
    
    joblib.dump(best_model, model_path)
    joblib.dump(feature_scaler, fscaler_path)
    joblib.dump(target_transformer, tscaler_path)
    print(f"模型已保存: {model_path}")

    # --- 7. 新增功能：生成全周期预测 ---
    print("生成全周期（至2050年）预测...")
    
    # 准备全周期的 X 特征
    # 必须填充特征中的NaN（例如，如果 Planetary_Scaled 最终为NaN）
    X_full = data[feature_names].fillna(0)    
    
    X_full_scaled = feature_scaler.transform(X_full)
    Y_pred_full_transformed = best_model.predict(X_full_scaled)
    
    # 反变换回原始尺度
    Y_pred_full_raw = target_transformer.inverse_transform(Y_pred_full_transformed.reshape(-1, 1)).flatten()
    
    # 创建预测列和二次残差列
    pred_col_name = f'LGBM_Pred_{model_name}'
    resid_col_name = f'LGBM_Resid_{model_name}'
    
    data[pred_col_name] = Y_pred_full_raw
    # 原始残差 - LGBM预测的残差 = 二次残差
    data[resid_col_name] = data[target_residual_column] - data[pred_col_name]
    
    # 准备统计字典
    stats_dict = {
        'model': model_name,
        # 新增 M_CV_R2 和 M_T_R2
        'M_T_R2': final_r2_test,
        'M_CV_R2': best_cv_score,
        'n_features': len(feature_names),
        'top_20_mod_count': len(mod_features_in_top),
        'best_params': opt_lgbm.best_params_,
        'training_time': time.time() - start_time
    }

    # 返回包含新列的DataFrame（仅含必要列）和统计结果
    return data[['日期', pred_col_name, resid_col_name]], stats_dict


# 主执行 - 运行 LGBM 全频率梳版
if __name__ == "__main__":
    model_splits = {
        'M8+2': '1996-08-01', 
        'M8+3': '1986-08-01',
        'M0+3': '1986-09-01', 
        'M0+2': '1996-08-01'
    }
    
    print("=== 运行 LGBM (全频率梳版: B0 + 22y, QBO, 1y, 154d, 25-30d) ===")
    print("策略: 物理全频率梳 + 11年调制 + 目标稳定 + 强正则化")
    print("新增: 保存模型和全周期二次残差")
    
    # --- 新增：在循环外加载和准备数据 ---
    try:
        data_source_file = "综合分析结果/汇总拟合1结果.csv"
        print(f"\n加载源数据: {data_source_file}")
        full_data_to_save = pd.read_csv(data_source_file, parse_dates=['日期'])
    except Exception as e:
        print(f"致命错误：无法读取 {data_source_file}。错误: {e}")
        exit()

    # 检查并扩展日期至2050年
    max_date_in_file = full_data_to_save['日期'].max()
    target_date = pd.to_datetime('2050-12-31')
    
    if max_date_in_file < target_date:
        print(f"警告: 源数据最大日期为 {max_date_in_file}。")
        print(f"警告: 正在扩展日期范围至 {target_date}。")
        print("警告: 请确保 '拟合SSN' (行星拟合) 列在源文件中已外推至2050年，否则预测将不准确！")
        
        all_dates_df = pd.DataFrame({'日期': pd.date_range(
            start=full_data_to_save['日期'].min(), 
            end=target_date, 
            freq='D'
        )})
        full_data_to_save = pd.merge(all_dates_df, full_data_to_save, on='日期', how='left')

    # 用于保存统计结果
    results_stats_list = []
    
    # 循环处理每个模型
    for model_name, split_date in model_splits.items():
        print(f"\n{'='*50}")
        print(f"处理模型: {model_name} (训练结束于 {split_date})")
        print(f"{'='*50}")
        
        # 传入完整数据
        result_df, stats_dict = run_lgbm_full_comb_validation(
            data_full=full_data_to_save.copy(), # 传入数据的副本
            model_name=model_name,
            train_start_date='1855-12-02',
            train_end_date=split_date,
            test_end_date='2019-11-30'
        )
        
        if result_df is not None and stats_dict is not None:
            results_stats_list.append(stats_dict)
            # 将该模型的结果合并回主DataFrame
            full_data_to_save = pd.merge(full_data_to_save, result_df, on='日期', how='left')
    
    # --- 新增：保存包含所有结果的CSV ---
    if results_stats_list:
        output_filename = "综合分析结果/汇总拟合2_LGBM结果.csv"
        print(f"\n{'='*60}")
        print(f"保存所有模型的二次拟合和残差到: {output_filename}")
        try:
            full_data_to_save.to_csv(output_filename, index=False, float_format='%.6f')
            print(f"保存成功。文件包含 {len(full_data_to_save)} 行数据。")
        except Exception as e:
            print(f"保存CSV失败: {e}")
        
        # --- 打印总结报告 (使用统计数据) ---
        results_to_print = []
        for r in results_stats_list:
            results_to_print.append({
                'model': r['model'],
                # 修正: 使用新的键名 M_T_R2
                'M_T_R2': r['M_T_R2'], 
                # 修正: 加入 M_CV_R2 方便查看
                'M_CV_R2': r['M_CV_R2'], 
                'n_features': r['n_features'],
                'top_20_mod': r['top_20_mod_count'],
                'time(s)': r['training_time']
            })
            
        results_df = pd.DataFrame(results_to_print)
        print("\n" + "="*60)
        print("LGBM 全频率梳版 总结")
        print("="*60)
        
        # 打印关键列
        print(results_df[['model', 'M_CV_R2', 'M_T_R2', 'n_features', 'top_20_mod', 'time(s)']].to_string(index=False, float_format="%.4f"))
        
        # 使用 M_T_R2 计算平均值
        avg_r2 = results_df['M_T_R2'].mean()
        print(f"\n平均 Test R² (M_T_R2): {avg_r2:.4f}")
        
        # 使用 M_T_R2 检查正负值
        positive_r2 = results_df[results_df['M_T_R2'] > 0]
        if len(positive_r2) == len(results_df):
            print("✅✅✅ 巨大成功: 所有模型 Test R² 均为正值！")
        elif len(positive_r2) > 0:
            print(f"✅ 成功: {len(positive_r2)} 个模型获得正 Test R²。")
        else:
            print("❌ 失败: 所有模型 Test R² 仍为负值。")
        
        if results_df['top_20_mod'].mean() >= 10:    
             print("💡 洞察: 调制特征 (Mod_) 在模型中占有重要地位。")

    else:
        print("\n所有模型处理失败")

LightGBM 导入成功。
=== 运行 LGBM (全频率梳版: B0 + 22y, QBO, 1y, 154d, 25-30d) ===
策略: 物理全频率梳 + 11年调制 + 目标稳定 + 强正则化
新增: 保存模型和全周期二次残差

加载源数据: 综合分析结果/汇总拟合1结果.csv

处理模型: M8+2 (训练结束于 1996-08-01)

=== LGBM 最终全频率梳版: M8+2 ===
1. 生成 B0 星历特征 + 全频率梳特征 (全周期)...
2. 生成调制特征 (全周期)...
... 正在添加 'Planetary_Scaled_Diff' (速度) 特征...
... 正在添加 'Planetary_Scaled_Accel' (加速度) 特征...
基础特征数量 (包括 Diff 和 Accel): 32
总特征数: 64 (全频率梳 + 速度 + 加速度)
应用 QuantileTransformer 稳定目标变量 Y (残差)...
数据: 训练51378, 测试8521
训练LGBM (全频率梳版)...

LGBM 最佳 CV R² (M_CV_R2): 0.0629
LGBM 检验集 Test R² (M_T_R2): 0.0108
✅ 成功！模型R²为正值。
最佳参数: OrderedDict({'colsample_bytree': 1.0, 'learning_rate': 0.0021322706054339017, 'n_estimators': 675, 'num_leaves': 14, 'reg_alpha': 0.1, 'reg_lambda': 20.0, 'subsample': 0.7})

特征重要性 (Top 20):
Mod_Hale_22y_sin        2179
Hale_22y_cos            1587
Mod_Hale_22y_cos        1093
Hale_22y_sin            1016
Mod_QBO_2.7y_cos         774
Mod_QBO_2.2y_sin         294
Mod_QBO_2.7y_sin         259
QBO_2.7y_cos             178
Mod_Ann